In [31]:
import pandas as pd
from scipy import stats
import numpy as np
import requests

In [86]:
repos = pd.read_csv("repos.csv", )
results = []

# Get results
for url in repos["Git"]:
    url_master = url.replace("github.com", "raw.githubusercontent.com") + "/refs/heads/master/answer_associativity.txt"
    response = requests.get(url_master)
    if response.status_code == 200:
        results.append(response.text.strip())
    else:
        url_main = url.replace("github.com", "raw.githubusercontent.com") + "/refs/heads/main/answer_associativity.txt"
        response = requests.get(url_main)
        if response.status_code == 200:
            results.append(response.text.strip())
        else:
            print("Foiré noob", url_master)
            results.append("")

repos["results"] = results

Foiré noob https://raw.githubusercontent.com/PaulineRoches/REP_popo_sos.git/refs/heads/master/answer_associativity.txt
Foiré noob https://raw.githubusercontent.com/l55I1/REP/refs/heads/master/answer_associativity.txt
Foiré noob https://raw.githubusercontent.com/Humilokaki/TP2-REP-INSA-202425.git/refs/heads/master/answer_associativity.txt


In [87]:
repos

,Binome1,Binome2,Sujet,Git,NRep,results
0,Matys Couroussé,Axel Homery,Sujet 2,https://github.com/Matys53/REP_TP,1000000,93.08
1,Solène Lemonnier,Pauline Roches,Sujet 2,https://github.com/PaulineRoches/REP_popo_sos.git,50000,
2,Chenyu Li,Bohan Ha,Sujet 3,https://github.com/l55I1/REP,1000000,
3,Victor van Kempen,Matthieu Hillairet,Sujet 1,https://github.com/matth1446/rep_mh_vvk,100,79.13
4,Antoine MARCHAL DOMBRAT,Jean Haurogné 🤓,Sujet 1,https://github.com/AntoineMarchalDombrat/REP_A...,100000,77.3
5,Ezgi Ozel,Alp Jakop,Sujet 2,https://github.com/alpjakop/REP,1000000,100.00
6,Noé Bourmalo,Aidan Gallagher,Sujet 2,https://github.com/Humilokaki/TP2-REP-INSA-202...,512,
7,Jeremy Brunet,Arno Lecrivain,Sujet 2,https://github.com/Mistyycs/REP-TP2,1000,83.0
8,Noa Berton,Guillaume Freyermuth,Sujet 1,https://github.com/Kaeios/assoc-REP/,100,100.00
9,Alexis Simon,Malvin Chevallier,Sujet 3,https://github.com/Dyskal/repro,125000,78.48


In [88]:
# Parse floats
repos = repos[pd.to_numeric(repos["results"], errors="coerce").notna()]
repos["results"] = repos["results"].astype("float") / 100.0
repos.reset_index(inplace=True, drop=True)
print(repos)
repos.dtypes

                   Binome1               Binome2    Sujet  \
0          Matys Couroussé           Axel Homery  Sujet 2   
1        Victor van Kempen    Matthieu Hillairet  Sujet 1   
2  Antoine MARCHAL DOMBRAT       Jean Haurogné 🤓  Sujet 1   
3                Ezgi Ozel             Alp Jakop  Sujet 2   
4            Jeremy Brunet        Arno Lecrivain  Sujet 2   
5               Noa Berton  Guillaume Freyermuth  Sujet 1   
6             Alexis Simon     Malvin Chevallier  Sujet 3   

                                                 Git     NRep  results  
0                  https://github.com/Matys53/REP_TP  1000000   0.9308  
1            https://github.com/matth1446/rep_mh_vvk      100   0.7913  
2  https://github.com/AntoineMarchalDombrat/REP_A...   100000   0.7730  
3                    https://github.com/alpjakop/REP  1000000   1.0000  
4                https://github.com/Mistyycs/REP-TP2     1000   0.8300  
5               https://github.com/Kaeios/assoc-REP/      100   1.0000  


/tmp/ipykernel_55855/3125347725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repos["results"] = repos["results"].astype("float") / 100.0


Binome1     object
Binome2     object
Sujet       object
Git         object
NRep         int64
results    float64
dtype: object

In [89]:
# Compute mean2mean stats
p_values = np.ones((len(repos), len(repos)))

for i in range(len(repos)):
    for j in range(i, len(repos)):
        sd1 = np.sqrt(repos["results"][i]*(1-repos["results"][i])) * ((repos["NRep"][i] + 1) / repos["NRep"][i]) if repos["results"][i] != 1.0 else 0.01
        sd2 = np.sqrt(repos["results"][j]*(1-repos["results"][j])) * ((repos["NRep"][i] + 1) / repos["NRep"][i]) if repos["results"][i] != 1.0 else 0.01
        t_stat, p_val = stats.ttest_ind_from_stats(
            mean1=repos["results"][i], std1=sd1, nobs1=repos["NRep"][i],
            mean2=repos["results"][j], std2=sd2, nobs2=repos["NRep"][j],
            equal_var=False
        )
        p_values[i, j] = p_val
        p_values[j, i] = p_val

df_p_values = pd.DataFrame(p_values)
df_p_values

,0,1,2,3,4,5,6
0,1.000000e+00,0.000874,0.000000e+00,0.000000,7.758388e-17,0.000000e+00,0.000000e+00
1,8.737414e-04,1.000000,6.568407e-01,0.000002,3.673254e-01,1.744649e-06,8.745419e-01
2,0.000000e+00,0.656841,1.000000e+00,0.000000,2.120016e-06,0.000000e+00,2.153216e-11
3,0.000000e+00,0.000002,0.000000e+00,1.000000,0.000000e+00,1.000000e+00,0.000000e+00
4,7.758388e-17,0.367325,2.120016e-06,0.000000,1.000000e+00,2.518974e-42,1.637809e-04
5,0.000000e+00,0.000002,0.000000e+00,1.000000,2.518974e-42,1.000000e+00,3.386154e-134
6,0.000000e+00,0.874542,2.153216e-11,0.000000,1.637809e-04,3.386154e-134,1.000000e+00


In [97]:
df_p_values.iloc[[5]]

,0,1,2,3,4,5,6
5,0.0,0.000002,0.0,1.0,2.518974e-42,1.0,3.386154e-134
